In [1]:
from dataset.flame import FlameFOV
from dataset.flame import FlameThermal
from dataset.flame import FlameRGB
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

In [2]:
compose = transforms.Compose(
    [
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

In [ ]:
dataset = FlameThermal(download=True, transform=compose)
loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

In [ ]:
dataset = FlameFOV(download=True, transform=compose)
loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

In [ ]:
dataset = FlameRGB(download=True, transform=compose)
loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

In [ ]:
# Test the DataLoader
for images, bboxes in loader:
    print("Image batch shape:", images.shape)
    print("Bounding box batch shape:", bboxes.shape)
    break

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

image, bbox = next(iter(loader))

image = image[0].permute(1, 2, 0).numpy()

image = np.clip(image, 0, 1)

plt.imshow(image)
plt.title(f"Bounding Box: {bbox[0]}")
plt.show()